# <span style="color:#2834d4">전체 구조</span>

<img src="img/affine_ex.png" width="70%" height="70%" align="left">
<img src="img/cnn_ex.png" width="70%" height="70%" align="left">

- CNN에서는 convolutional layer와 pooling layer가 새롭게 등장함
- 출력과 가까운 층에서는 Affine-ReLU의 구성을 사용할 수 있음

# <span style="color:#2834d4">합성곱 계층</span>

- 합성곱 계층의 입출력 데이터를 특징 맵(feature map)이라고 함

## 1. 완전연결 계층의 문제점

- 데이터의 형상이 무시됨
    - ex) 이미지의 경우 3차원을 1차원으로 평탄화한 후 입력하기 때문에, 공간적 정보를 살릴 수 없게됨.
- 그러나 합성곱 계층은 형상을 유지함
    - ex) 이미지의 경우 3차원 데이터로 입력받고, 다음 계층에서도 3차원 데이터로 전달하여 이미지처럼 형상을 가진 데이터를 제대로 이해할 수 있음

## 2. 합성곱 (convolution) 연산 

<img src="img/conv_ex.png" width="40%" height="40%" align="left">

- 필터(=커널)의 윈도우를 일정 간격으로 이동해가며 입력데이터에 적용함
- 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구하고, 그 결과를 출력의 해당 장소에 저장함

<img src="img/conv_bias.png" width="70%" height="70%" align="left">

- CNN에서는 필터의 매개변수가 그동안의 가중치에 해당하고, 편향도 존재함
    - 편향은 항상 하나(1x1)만 존재

## 3. 패딩 (padding)

<img src="img/padding.png" width='70%' heigth='70%' align='left'>

- 합성곱 연산을 하기 전에 입력 데이터 주변을 특정 값(예컨대 0)으로 채우는 것
- 주로 출력 크기를 조정할 목적으로 사용함
    - 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달할 수 있음

## 4. 스트라이드 (stride)

- 필터를 적용하는 간격을 지정

### ※출력 크기 계산
## $O = \frac{I \ + \ 2P \ - \ F}{S} + 1$

- 입력크기-$I$, 필터크기-$F$, 출력크기-$O$, 패딩-$P$, 스트라이드-$S$
- 단, $\frac{I \ + \ 2P \ - \ F}{S}$ 가 정수로 나누어 떨어지는 값이어야 함
    - 정수가 아닐 경우 반올림이나 특별히 에러를 내지 않고 진행하도록 구현하는 경우도 있음

## 5. 3차원 데이터의 합성곱 연산

<img src="img/conv_ex_3d.png" width="50%" height="50%" align="left">

- 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함
- 모든 채널의 필터가 같은 크기여야 함

## 6. 블록으로 생각하기

- 채널수 $C$, 높이 $H$, 너비 $W$, 필터 $F$ 데이터 수 $N$
- 3차원 데이터 $\to \ (C, H, W)$, 필터도 같은 순서로 $(C, FH, FW)$

**하나의 필터만 사용한 경우**

<img src="img/conv_block_f1.png" width="50%" height="50%" align="left">

**여러 필터를 사용한 경우**

<img src="img/conv_block_fn.png" width="50%" height="50%" align="left">

**bias 추가한 경우**

<img src="img/conv_block_bias.png" width="60%" height="60%" align="left">

**배치 처리한 경우**

<img src="img/conv_block_batch.png" width="60%" height="60%" align="left">

# <span style="color:#2834d4">풀링 계층</span>
- 세로 가로 방향의 공간을 줄이는 연산
- 종류: max pooling(대상 영역에서 최댓값을 취하는 연산), average pooling(대상 영역의 평균 계산) 등
- 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통임

max pooling 예시

<img src="img/max_pool.png" width ="50%" height="50%" align="left">

## 1. 풀링 계층의 특징

- 학습해야할 매개변수가 없음
- 채널 수가 변하지 않음
    - 채널마다 독립적으로 계산하기 때문
- 입력의 변화에 영향을 적게 받음

# <span style="color:#2834d4">합성곱/풀링 계층 구현하기</span>

## 1. im2col로 데이터 전개하기

<img src="img/im2col.png" width="40%" height="40%" align="left">

- 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는 함수
    - 입력 데이터에서 필터를 적용하는 영역을 한 줄로 늘어놓는데, 이 전개를 필터를 적용하는 모든 영역에서 수행
- 배치 안의 데이터 수까지 포함한 4차원 데이터를 2차원으로 변환
- 필터 적용 영역이 겹치게 되면 원소 수가 원래 블록의 원소 수보다 많아져서 메모리를 더 많이 소비한다는 단점이 있음
    - 그러나 컴퓨터는 큰 행렬을 묶어서 계산하는데 탁월. 선형대수 라이브러리를 활용해 효율을 높일 수 있음
    
<img src="img/im2col_.png" width="60%" height="60%" align="left">

In [1]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape  # (이미지 수, 채널 수, 높이, 너비)
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')  # 각 axis의 edge에 pad만큼 패딩
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col  # 2차원 배열

In [2]:
import numpy as np

x1 = np.random.rand(1, 3, 7, 7)  # (데이터수, 채널수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)  # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


## 2. 합성곱 계층 구현하기

In [3]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T  # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W) 로 축 순서 변경
        
        return out

## 3. 풀링 계층 구현하기

In [4]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)  # 지정한 차원의 축마다 최댓값 구함
        
        out = np.reshpae(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

# <span style="color:#2834d4">CNN 구현하기</span>

단순한 CNN 네트워크 구성

<img src='img/simpleCNN.png' width='50%' height='50%' align='left'>

In [5]:
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *

class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28),
                conv_param={'filter_num':30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : 
            t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 10

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

=== epoch:1, train acc:0.455, test acc:0.409 ===<br>
=== epoch:2, train acc:0.809, test acc:0.794 ===<br>
=== epoch:3, train acc:0.868, test acc:0.863 ===<br>
=== epoch:4, train acc:0.896, test acc:0.897 ===<br>
=== epoch:5, train acc:0.918, test acc:0.903 ===<br>
=== epoch:6, train acc:0.927, test acc:0.913 ===<br>
=== epoch:7, train acc:0.929, test acc:0.918 ===<br>
=== epoch:8, train acc:0.942, test acc:0.924 ===<br>
=== epoch:9, train acc:0.948, test acc:0.929 ===<br>
=== epoch:10, train acc:0.957, test acc:0.939 ===<br>
=============== Final Test Accuracy ===============<br>
test acc:0.945<br>

<img src='img/train_simpleCNN.png' width='50%' height='50%' align='left'>

# <span style="color:#2834d4">CNN 시각화하기</span>


## 층 깊이에 따른 추출 정보 변화

<img src='img/cnn_visualizing.png' width='70%' height='70%' align='left'>

- 계층이 깊어질수록 추출되는 정보가 추상화됨
- 층이 깊어질수록 뉴런이 반응하는 대상이 단순한 모양에서 고급정보로 변화
    - 다시말하면, 사물의 의미를 이해하도록 변화하는 것
    - 초반에는 단순한 에지에 반응하고, 이어서 텍스처에 반응하고, 더 복잡한 사물의 일부에 반응하도록 변화

# <span style="color:#2834d4">대표적인 CNN</span>

## LeNet

<img src='img/LeNet.png' width='70%' height='70%' align='left'>

- activation function으로 sigmoid 사용
- subsampling을 통해 feature map의 크기를 줄임 (max pooling 사용 X)

## AlexNet

<img src='img/AlexNet.png' width='70%' height='70%' align='left'>

- activation function으로 ReLU 사용
- max pooling을 통해 feature map의 크기를 줄임
- LRN(Local Response Normalization) 이라는 국소적 정규화를 실시하는 계층을 이용
- dropout 사용

# <span style="color:#2834d4">정리</span>

- CNN은 지금까지의 완전 연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가한 것.
- 합성곱 계층과 풀링 계층은 이미지를 행렬로 전개하는 함수를 이용하면 간단하고 효율적으로 구현 가능.
- CNN을 시각화해보면 계층이 깊어질수록 고급 정보가 추출되는 모습을 확인할 수 있음.
- 대표적인 CNN에는 LeNet과 AlexNet이 있음.
- 딥러닝의 발전에는 빅데이터와 GPU가 크게 기여함.